In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("../data/data_movements_dhruvaTest.csv")

In [5]:
df.loc[df.state == 'reach'].trial.value_counts()

trial
95     81
274    69
124    26
20     26
19     23
       ..
59      9
60      9
5       9
4       9
41      9
Name: count, Length: 300, dtype: int64

In [6]:
df.loc[(df.state == 'reach') & (df.trial == 95)].iloc[0]

Unnamed: 0           158
trial                 95
state              reach
sample             27550
time          459.196992
x               0.074821
y                0.37573
Name: 27455, dtype: object

In [9]:
df.loc[(df.state == 'reach') & (df.trial == 95)].iloc[-1]

Unnamed: 0           238
trial                 95
state              reach
sample             27630
time          460.529585
x              -0.813019
y               9.823062
Name: 27535, dtype: object

In [17]:
df.loc[(df.trial == 95)].to_csv('trial_95_error_check.csv', index=False)